In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [2]:
data_file = '/Users/nevinaresh/Documents/CPET Research/BxBData.json'
outcomes_file = '/Users/nevinaresh/Documents/CPET Research/CPX database_outcomes_meds_10_22.xlsx'

In [151]:
#reads in the data from the json file
raw_data = pd.read_json(data_file).transpose()
raw_data.head(3)

,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,testTime,TestDuration,...,P_Diast,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain
1,[Margaret],[Raffin],[MRN-03907458],[Female],[12/26/1946],[157.5],[45.36],[10/11/2018],[4:11 PM],[15:30],...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,..."
2,[MYEDOE],[MYEDOE],[MRN-08/STANFORD CONCERT HF 08-7009-08],[Male],[5/1/1955],[175.3],[107.95],[6/19/2017],[2:34 PM],[26:23],...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.8, 22.8, 22.8, 22.8, 22.8, 22.8, 22.8, 22....","[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 6...","[634, 634, 634, 634, 634, 634, 634, 634, 634, ...","[756, 756, 756, 756, 756, 756, 756, 756, 756, ...","[1.73, 1.77, 1.61, 1.55, 0.84, 2.28, 1.5, 1.87...","[1.74, 2.26, 2.2, 2.13, 2.16, 1.69, 2.98, 2.28...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,..."
3,[Larry],[GERSTON],[MRN-27411321],[Male],[5/27/1946],[175.3],[83.5],[8/8/2014],[11:47 AM],[14:28],...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22....","[, , , , , , , , , , , , , , , , , , , , , , ,...","[649, 649, 649, 649, 649, 649, 649, 649, 649, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,..."


In [4]:
raw_data.shape

(4221, 54)

In [5]:
#formats columns and removes unnecessary
from src.jsonParsing import preprocess

preprocessed_data = preprocess(raw_data)
preprocessed_data.head(2)

,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,testTime,...,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test
0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,4:11 PM,...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00
1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,11:47 AM,...,"[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22....","[, , , , , , , , , , , , , , , , , , , , , , ,...","[649, 649, 649, 649, 649, 649, 649, 649, 649, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00


In [6]:
preprocessed_data.shape

(4220, 56)

In [7]:
#preprocessed_data[preprocessed_data['testDate'] == pd.to_datetime('10/11/2016', format='%m/%d/%Y')]

In [8]:
#filters those rows that have an ID1 of the form 'MRN-'MRNID
from src.jsonParsing import validmrn

data_with_mrn = validmrn(preprocessed_data)
data_with_mrn.head(2)

/Users/nevinaresh/Documents/Stanford/CPET-AI/src/jsonParsing.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_mrn['MRN'] = pd.to_numeric(valid_mrn['ID1'].str.split('-').str[1], errors='coerce')


,level_0,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,...,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test,MRN
0,0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,...,"[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00,3907458.0
1,1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,...,"[22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22....","[, , , , , , , , , , , , , , , , , , , , , , ,...","[649, 649, 649, 649, 649, 649, 649, 649, 649, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00,27411321.0


In [9]:
data_with_mrn.shape

(3882, 58)

In [10]:
preprocessed_data[preprocessed_data['testDate'] == pd.to_datetime("10/11/2016", format='%m/%d/%Y')]

,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,testTime,...,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test
2555,2557,Brianne,Purcell,MRN-35543081,Female,1/14/1980,177.8,113.4,2016-10-11,2:44 PM,...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[25.4, 25.4, 25.4, 25.4, 25.4, 25.4, 25.4, 25....","[41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...","[645, 645, 645, 645, 645, 645, 645, 645, 645, ...","[763, 763, 763, 763, 763, 763, 763, 763, 763, ...","[2.1, 2.24, 1.66, 2.35, 2.8, 4.67, 1.37, 3.23,...","[2.92, 4.7, 2.96, 3.45, 3.48, 7.03, 3.23, 4.92...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2016-10-11 14:44:00
2659,2661,HML-PHX_102-006_JB,HML-PHX_102-006_JB,MRN-HML-PHX_102-006_JB,Male,1/28/1980,172.7,90.26,2016-10-11,11:19 AM,...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...","[647, 647, 647, 647, 647, 647, 647, 647, 647, ...","[764, 764, 764, 764, 764, 764, 764, 764, 764, ...","[1.28, 1.41, 1.13, 1.01, 1.29, 1.24, 1.06, 1.2...","[2.23, 2.12, 1.73, 1.97, 2.18, 2.06, 2.11, 1.8...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2016-10-11 11:19:00


In [11]:
outcomes_data = pd.read_excel(outcomes_file, sheet_name="Outcomes_updated")
outcomes_data.head(2)

,ID,MRN,Test Date,MRN-test date,Death,death_date,death_yrs,pre_arrhythm_surg,pre_arrhythm_surg_date,pre_arrhythm_surg_yrs,...,post_stroke_yrs,post_valve_surgery_unspecified,post_valve_surgery_unspecified_date,post_valve_surgery_unspecified_yrs,post_ventricular_fib_flut,post_ventricular_fib_flut_date,post_ventricular_fib_flut_yrs,post_ventricular_tachycardia,post_ventricular_tachycardia_date,post_ventricular_tachycardia_yrs
0,1.0,4400,2016-04-29,4400-42489,0.0,NaT,NaN,NaN,NaT,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
1,2.0,84095,2012-09-07,84095-41159,0.0,NaT,NaN,0.0,NaT,NaN,...,NaN,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2014-03-31,1.56


In [12]:
outcomes_data[outcomes_data['Test Date'] == pd.to_datetime("10/11/2016", format='%m/%d/%Y')]

,ID,MRN,Test Date,MRN-test date,Death,death_date,death_yrs,pre_arrhythm_surg,pre_arrhythm_surg_date,pre_arrhythm_surg_yrs,...,post_stroke_yrs,post_valve_surgery_unspecified,post_valve_surgery_unspecified_date,post_valve_surgery_unspecified_yrs,post_ventricular_fib_flut,post_ventricular_fib_flut_date,post_ventricular_fib_flut_yrs,post_ventricular_tachycardia,post_ventricular_tachycardia_date,post_ventricular_tachycardia_yrs
3789,2467.0,34568683,2016-10-11,34568683-42654,0.0,NaT,NaN,0.0,NaT,NaN,...,NaN,0.0,NaT,NaN,0.0,NaT,NaN,0.0,NaT,NaN
3903,2549.0,35543081,2016-10-11,35543081-42654,0.0,NaT,NaN,0.0,NaT,NaN,...,NaN,NaN,NaT,NaN,0.0,NaT,NaN,1.0,2016-10-12,0.0


In [13]:
from src.outcomeParsing import preprocess

death_data = preprocess(outcomes_data)
death_data.head(2)

,MRN,Test Date,Death,death_date
0,4400,2016-04-29,0.0,NaT
1,84095,2012-09-07,0.0,NaT


In [129]:
composite_outcome_columns = [
    "post_cardiac_arrest",
    "post_cardiogenic_shock",
    "post_ecmo",
    "post_heart_lung_transplant",
    "post_htx",
    "post_iabp",
    "post_mcs_extracardiac",
    "post_mcs_implantable",
    "post_mcs_percutaneous",
    "post_renal_failure_requiring_dialysis",
    "post_shockable_rhythm",
    "post_ventricular_fib_flut"
]

In [135]:
outcome_copy = outcomes_data.copy().dropna(subset=composite_outcome_columns)
outcome_copy['composite_outcome'] = outcome_copy[composite_outcome_columns].max(axis=1)
outcome_copy.head(2)

,ID,MRN,Test Date,MRN-test date,Death,death_date,death_yrs,pre_arrhythm_surg,pre_arrhythm_surg_date,pre_arrhythm_surg_yrs,...,post_valve_surgery_unspecified,post_valve_surgery_unspecified_date,post_valve_surgery_unspecified_yrs,post_ventricular_fib_flut,post_ventricular_fib_flut_date,post_ventricular_fib_flut_yrs,post_ventricular_tachycardia,post_ventricular_tachycardia_date,post_ventricular_tachycardia_yrs,composite_outcome
1,2.0,84095,2012-09-07,84095-41159,0.0,NaT,NaN,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2014-03-31,1.56,0.0
2,3.0,124420,2017-05-26,124420-42881,0.0,NaT,NaN,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2022-03-31,4.85,0.0


In [138]:
from src.outcomeParsing import mergeDataOutcome

#alternative outcomes where we have more data (death, cardiovascular related hospitalization)
data_and_outcomes = mergeDataOutcome(data_with_mrn, death_data, 365*7)
data_and_outcomes

,level_0,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,...,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test,MRN,Test Date,Death,death_date
0,0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,...,"[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00,3907458.0,2016-06-16,0,NaT
1,0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,...,"[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00,3907458.0,2018-10-11,0,NaT
2,1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,...,"[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00,27411321.0,2013-06-12,0,NaT
3,1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,...,"[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00,27411321.0,2014-08-08,0,NaT
4,1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,...,"[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00,27411321.0,2016-08-10,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8021,4215,4217,Kara,NETTE,MRN-60452968,Female,7/1/1992,157.5,47.63,2018-05-30,...,"[759, 759, 759, 759, 759, 759, 759, 759, 759, ...","[1.03, 1.27, 1.01, 1.15, 1.44, 0.84, 1.2, 1.12...","[1.32, 1.56, 0.84, 0.9, 1.53, 1.64, 1.49, 1.34...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-05-30 10:08:00,60452968.0,2016-09-21,0,NaT
8022,4216,4218,Sophia,Zamoyski,MRN-74146523,Female,11/20/1999,185.4,70.31,2019-03-18,...,"[761, 761, 761, 761, 761, 761, 761, 761, 761, ...","[5.21, 0.68, 0.79, 1.66, 1.27, 0.97, 0.88, 0.9...","[1.79, 0.79, 1.64, 2.68, 1.63, 2.4, 1.21, 1.66...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2019-03-18 10:06:00,74146523.0,2019-03-18,0,NaT
8023,4217,4219,Adrian,HINOJOSA,MRN-23642655,Male,9/10/1993,177.8,57.2,2014-07-31,...,"[761, 761, 761, 761, 761, 761, 761, 761, 761, ...","[1.12, 1.14, 1.16, 1.55, 1.16, 1.29, 1.22, 1.5...","[1.75, 5.35, 2.36, 3.17, 2.66, 2.68, 2.56, 2.8...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-07-31 14:00:00,23642655.0,2011-05-11,0,NaT
8024,4218,4220,Jennifer,WOODS,MRN-25462326,Female,5/26/1969,167.6,69.9,2016-02-16,...,"[761, 761, 761, 761, 761, 761, 761, 761, 761, ...","[1.21, 1.15, 1.43, 1.64, 1.36, 1.29, 1.45, 1.1...","[1.97, 2.54, 2.92, 2.56, 2.03, 2.24, 1.91, 2.0...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2016-02-16 09:57:00,25462326.0,2012-03-27,0,NaT


In [155]:
data_and_outcomes = data_and_outcomes.merge(outcome_copy[['MRN', 'composite_outcome']], on='MRN', how='left').drop_duplicates(subset=['firstName', 'MRN', 'testDate'], keep='first')

In [156]:
data_and_outcomes['Death'].value_counts(), data_and_outcomes['composite_outcome'].value_counts()

(Death
 0    3693
 1     101
 Name: count, dtype: int64,
 composite_outcome
 0.0    2629
 1.0     671
 Name: count, dtype: int64)

In [157]:
x = np.array(data_and_outcomes['Time'][1]).astype(float)
y = np.array(data_and_outcomes['Rf'][1]).astype(float)
x_interp = np.arange(0, max(x), 5)
y_linear = interp1d(x, y, fill_value="extrapolate")
y_quadratic = interp1d(x, y, kind='quadratic', fill_value="extrapolate")
y_cubic = interp1d(x, y, kind='cubic', bounds_error=False, fill_value=y[[0]])
y_cubic(x_interp)
# plt.plot(x, y, 'o')
# plt.plot(x_interp, y_linear(x_interp), 'red')
# plt.plot(x_interp, y_quadratic(x_interp), 'green')
# plt.plot(x_interp, y_cubic(x_interp), 'yellow')
#plt.xticks(np.arange(0, max(data_and_outcomes['Time'][0]), 10))
#plt.yticks(np.arange(min(data_and_outcomes['Ti'][0]), max(data_and_outcomes['Ti'][0]), 10))

KeyError: 1

In [158]:
type(data_and_outcomes['Time'][0])

list

In [159]:
list(filter(None, np.array(data_and_outcomes['P_Diast'][0])))

[]

In [163]:
features = data_and_outcomes.iloc[:,19:-6]
time = data_and_outcomes['Time']
#spline to normalize

In [164]:
features['Power']

0        [, , , , , , , , , , , , , , , , , , , , , , ,...
4        [, , , , , , , , , , , , , , , , , , , , , , ,...
10       [, , , , , , , , , , , , , , , , , , , , , , ,...
65       [, , , , , , , , , , , , , , , , , , , , , , ,...
66       [, , , , , , , , , , , , , , , , , , , , , , ,...
                               ...                        
19185    [, , , , , , , , , , , , , , , , , , , , , , ,...
19194    [, , , , , , , , , , , , , , , , , , , , , , ,...
19195    [, , , , , , , , , , , , , , , , , , , , , , ,...
19196    [, , , , , , , , , , , , , , , , , , , , , , ,...
19197    [, , , , , , , , , , , , , , , , , , , , , , ,...
Name: Power, Length: 3794, dtype: object

In [165]:
features

,Rf,VT,VE,VO2,VCO2,RQ,O2exp,CO2exp,VE/VO2,VE/VCO2,...,P_Diast,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain
0,"[18.29, 27.03, 22.22, 17.6, 18.93, 17.05, 17.7...","[0.452, 0.655, 0.508, 0.549, 0.473, 0.52, 0.50...","[8.268, 17.703, 11.289, 9.66, 8.953, 8.864, 8....","[189.0858298, 610.0869062, 322.55929, 278.3199...","[138.4675518, 442.0395832, 236.8627716, 206.59...","[0.73, 0.72, 0.73, 0.74, 0.73, 0.75, 0.75, 0.7...","[82.3, 110.5, 89, 96, 83.5, 92.2, 90.6, 80.2, ...","[9.5, 20.4, 13.3, 14.7, 11.9, 13, 12, 11.2, 20...","[37, 25.9, 30.2, 30.3, 31, 32.2, 34.2, 32.3, 2...","[50.5, 35.8, 41.1, 40.8, 42.3, 43.2, 45.4, 42....",...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,..."
4,"[15.5, 14.81, 15.08, 16.44, 7.52, 15.31, 10.51...","[0.974950132723634, 0.570080673841539, 0.45382...","[15.116, 8.446, 6.842, 9.22, 18.05, 9.834, 5.6...","[497.4793992, 244.6065718, 215.5366443, 328.20...","[380.230353, 180.1116693, 151.3911278, 224.805...","[0.76, 0.74, 0.7, 0.68, 0.78, 0.76, 0.73, 0.68...","[166.638966199834, 100.146551290231, 78.526318...","[29.9827760481954, 14.8894057926413, 12.339850...","[28.2, 30.3, 26.8, 24.6, 28.9, 29.7, 28.3, 26....","[36.9, 41.1, 38.2, 35.9, 36.8, 39, 38.9, 39.1,...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22....","[, , , , , , , , , , , , , , , , , , , , , , ,...","[649, 649, 649, 649, 649, 649, 649, 649, 649, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,..."
10,"[14.96, 12.82, 20.27, 13.73, 15.62, 13.33, 11....","[0.646496646655018, 0.605708214689401, 0.29979...","[9.673, 7.765, 6.077, 4.9, 6.341, 5.697, 4.832...","[384.0084722, 322.5982405, 236.186713, 178.567...","[319.77292, 263.418661, 180.1368256, 137.04008...","[0.83, 0.82, 0.76, 0.77, 0.75, 0.75, 0.74, 0.7...","[104.622327991806, 96.7705548384246, 48.946118...","[26.1045964579944, 25.084885658854, 10.9109055...","[22.5, 21.3, 19.7, 22.1, 20.8, 20.9, 19.5, 19....","[27, 26.1, 25.9, 28.7, 27.6, 27.8, 26.3, 26.2,...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[NONE, REST, REST, REST, REST, REST, REST, RES...","[25.4, 25.4, 25.4, 25.4, 25.5, 25.5, 25.5, 25....","[, , , , , , , , , , , , , , , , , , , , , , ,...","[639, 639, 639, 639, 639, 639, 639, 639, 639, ...","[768, 768, 768, 768, 768, 768, 768, 768, 768, ...","[1.27, 1.12, 1.23, 1.95, 1.24, 1.56, 1.89, 1.5...","[2.74, 3.56, 1.73, 2.42, 2.6, 2.94, 3.2, 3.04,...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,..."
65,"[7.41, 9.79, 25.21, 17.19, 16.71, 19.17, 17.8,...","[1.52036827690762, 1.03805158007509, 0.2335106...","[11.262, 10.16, 5.887, 8.082, 13.378, 13.038, ...","[393.7454097, 328.9288157, 139.2203353, 191.06...","[307.6961611, 259.5268188, 103.0570547, 141.09...","[0.78, 0.79, 0.74, 0.74, 0.75, 0.73, 0.72, 0.7...","[257.269561545132, 178.446980857702, 42.419396...","[50.57697286024, 32.3243959610808, 5.098485167...","[27.3, 28.8, 29.6, 36, 28.8, 30.3, 30.2, 32.8,...","[34.9, 36.5, 40, 48.7, 38.4, 41.4, 41.6, 44.8,...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.8, 22.8, 22.8, 22.8, 22.8, 22.8, 22.8, 22....","[, , , , , , , , , , , , , , , , , , , , , , 

In [166]:
from src.preprocessing import feature_pruning
pruned_features = feature_pruning(features, time)
pruned_features

,Rf,VT,VE,VO2,VCO2,RQ,O2exp,CO2exp,VE/VO2,VE/VCO2,...,Grade,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,interpolated_time,time_start_at_zero
0,"[18.29, 26.75180308914143, 17.6, 18.1965086571...","[0.452, 0.6209065368829562, 0.549, 0.480135428...","[8.268, 16.642064796540417, 9.66, 8.7045635076...","[189.0858298, 559.7813181269902, 278.319911399...","[138.4675518, 406.25985713515706, 206.59725759...","[0.73, 0.7211426478591703, 0.74, 0.73839956118...","[82.3, 105.23671071014239, 95.99999999999999, ...","[9.5, 18.93893467019352, 14.699999999999998, 1...","[37.0, 26.217279142639605, 30.3, 31.4702629002...","[50.5, 36.14119972906257, 40.8, 42.73582409085...",...,"[0.0, 3.324374028376501e-52, -1.05057568862826...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.099999999999998, 22.1, 22.1000000000...","[52.0, 51.99999999999999, 51.99999999999999, 5...","[639.0, 639.0269907884979, 638.9999999999999, ...","[755.0, 755.0, 755.0, 755.0, 755.0, 755.0, 755...","[1.07, 0.8518315233722866, 1.1199999999999999,...","[2.21, 1.351790619599702, 2.29, 1.974617407259...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",False
4,"[15.5, 14.809999999999999, 16.276026006845886,...","[0.974950132723634, 0.570080673841539, 0.38467...","[15.116, 8.446, 7.295059535106092, 14.38249432...","[497.4793992, 244.6065718, 249.22407477786766,...","[380.230353, 180.1116693, 168.75227230275695, ...","[0.76, 0.7400000000000001, 0.6815828542592595,...","[166.638966199834, 100.14655129023102, 65.4703...","[29.9827760481954, 14.8894057926413, 10.604808...","[28.2, 30.3, 25.17004030261357, 25.62370227604...","[36.9, 41.10000000000001, 36.73956952286325, 3...",...,"[0.0, 8.331010251020017e-92, 1.116051895098710...","[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.5, 22.5, 22.5, 22.500000000000004, 22.5, 2...",NaN,"[649.0, 649.0, 649.0, 649.0000000000001, 649.0...","[760.0, 760.0, 760.0, 760.0000000000001, 760.0...","[1.45, 1.54, 1.5856177229154358, 1.83605408301...","[2.42, 2.5100000000000002, 1.972224460368583, ...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",True
10,"[14.96, 9.017432510918821, 19.731711659387447,...","[0.646496646655018, 0.743277944013596, 0.34857...","[9.673, 8.52520226118146, 6.5324207147679125, ...","[384.0084722, 355.1094357117334, 259.741065269...","[319.77292, 297.258477690566, 200.852681206289...","[0.83, 0.8484884866275205, 0.7687854533594306,...","[104.622327991806, 118.86172203697642, 56.3463...","[26.1045964579944, 31.043683077847035, 13.3512...","[22.5, 22.644854622200352, 19.647874696310875,...","[27.0, 26.891156408232835, 25.561451283400327,...",...,"[0.0, -1.0099561327475258e-49, 6.7330408849835...","[NONE, REST, REST, REST, REST, REST, REST, RES...","[25.4, 25.395378491149835, 25.403081005900106,...",NaN,"[639.0, 639.0, 638.9999999999999, 638.99999999...","[768.0, 768.0, 767.9999999999999, 767.99999999...","[1.27, 1.264874991441299, 1.1011944501502453, ...","[2.74, 4.247516775903432, 1.9938777049532665, ...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",False
65,"[7.41, -3.815142060961671, 9.79, 20.6697039138...","[1.52036827690762, 1.9053424941556572, 1.03805...","[11.262, 13.891989190334579, 10.16, 6.75452522...","[393.7454097, 489.1582709505118, 328.9288157, ...","[307.6961611, 387.46527587523343, 259.5268188,...","[0.78, 0.8126927088270169, 0.7900000000000001,...","[257.269561545132, 324.5498997696384, 178.4469...","[50.57697286024, 61.78172852848574, 32.3243959...","[27.3, 30.000140400163733, 28.8, 34.6295441762...","[34.9, 37.010506887771825, 36.5, 47.0361569950...",...,"[0.0, -4.737263016582633e-21, 0.0, -5.44785246...","[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.8, 22.799999999999997, 22.799999999999997,...",NaN,"[635.0, 634.3915860176253, 635.0000000000001, ...","[769.0, 768.9999999999999, 768.9999999999999, ...","[2.22, 3.863958208204648, 2.73, 1.049237124620...","[5.88, 6.221571372229379, 3.4, 1.8649613990514...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,..."

In [167]:
pruned_features['outcome'] = data_and_outcomes['composite_outcome']
pruned_features.head(2)

,Rf,VT,VE,VO2,VCO2,RQ,O2exp,CO2exp,VE/VO2,VE/VCO2,...,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,interpolated_time,time_start_at_zero,outcome
0,"[18.29, 26.75180308914143, 17.6, 18.1965086571...","[0.452, 0.6209065368829562, 0.549, 0.480135428...","[8.268, 16.642064796540417, 9.66, 8.7045635076...","[189.0858298, 559.7813181269902, 278.319911399...","[138.4675518, 406.25985713515706, 206.59725759...","[0.73, 0.7211426478591703, 0.74, 0.73839956118...","[82.3, 105.23671071014239, 95.99999999999999, ...","[9.5, 18.93893467019352, 14.699999999999998, 1...","[37.0, 26.217279142639605, 30.3, 31.4702629002...","[50.5, 36.14119972906257, 40.8, 42.73582409085...",...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.099999999999998, 22.1, 22.1000000000...","[52.0, 51.99999999999999, 51.99999999999999, 5...","[639.0, 639.0269907884979, 638.9999999999999, ...","[755.0, 755.0, 755.0, 755.0, 755.0, 755.0, 755...","[1.07, 0.8518315233722866, 1.1199999999999999,...","[2.21, 1.351790619599702, 2.29, 1.974617407259...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",False,0.0
4,"[15.5, 14.809999999999999, 16.276026006845886,...","[0.974950132723634, 0.570080673841539, 0.38467...","[15.116, 8.446, 7.295059535106092, 14.38249432...","[497.4793992, 244.6065718, 249.22407477786766,...","[380.230353, 180.1116693, 168.75227230275695, ...","[0.76, 0.7400000000000001, 0.6815828542592595,...","[166.638966199834, 100.14655129023102, 65.4703...","[29.9827760481954, 14.8894057926413, 10.604808...","[28.2, 30.3, 25.17004030261357, 25.62370227604...","[36.9, 41.10000000000001, 36.73956952286325, 3...",...,"[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.5, 22.5, 22.5, 22.500000000000004, 22.5, 2...",NaN,"[649.0, 649.0, 649.0, 649.0000000000001, 649.0...","[760.0, 760.0, 760.0, 760.0000000000001, 760.0...","[1.45, 1.54, 1.5856177229154358, 1.83605408301...","[2.42, 2.5100000000000002, 1.972224460368583, ...","[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...",True,0.0


In [168]:
time_series_features = pd.DataFrame()
for col in pruned_features.iloc[:,0:-3]:
    for idx, row in pruned_features.iterrows():
        try:
            time_series_features.at[idx, col + '_mean'] = np.mean(row[col])
        except:
            time_series_features.at[idx, col + '_mean'] = np.nan
        try:
            time_series_features.at[idx, col + '_max'] = np.max(row[col])
        except:
            time_series_features.at[idx, col + '_max'] = np.nan
        try:
            time_series_features.at[idx, col + '_min'] = np.min(row[col])
        except:
            time_series_features.at[idx, col + '_min'] = np.nan
        try:
            time_series_features.at[idx, col + '_std'] = np.std(row[col])
        except:
            time_series_features.at[idx, col + '_std'] = np.nan
time_series_features

/var/folders/yz/gdzsf62s3rjg09h2ns2j9kv00000gn/T/ipykernel_66732/480083839.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  time_series_features.at[idx, col + '_mean'] = np.mean(row[col])
/var/folders/yz/gdzsf62s3rjg09h2ns2j9kv00000gn/T/ipykernel_66732/480083839.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  time_series_features.at[idx, col + '_max'] = np.max(row[col])
/var/folders/yz/gdzsf62s3rjg09h2ns2j9kv00000gn/T/ipykernel_66732/480083839.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the

,Rf_mean,Rf_max,Rf_min,Rf_std,VT_mean,VT_max,VT_min,VT_std,VE_mean,VE_max,...,PB_min,PB_std,Ti_mean,Ti_max,Ti_min,Ti_std,Te_mean,Te_max,Te_min,Te_std
0,26.343108,50.043451,6.693317,7.409566,0.817306,1.412597,0.212274,0.229081,21.862376,53.535141,...,755.0,9.504415e-14,0.995927,1.650559,0.137207,0.219165,1.472634,5.120000,0.297896,0.688078
4,19.614454,39.811840,6.092655,6.364680,1.437693,2.867007,0.182088,0.818760,31.405264,101.433000,...,760.0,9.441178e-14,1.377040,2.725223,0.710000,0.324599,2.017309,6.310680,0.771320,0.892687
10,58.432143,430.980933,-99.208942,117.091794,-0.112978,12.462207,-18.777080,5.948967,-57.271659,222.796267,...,768.0,1.701798e-12,1.103423,12.406791,-2.452976,2.030230,3.013339,16.726619,-4.910225,4.298140
65,28.868194,273.182742,-80.721037,77.819433,1.109187,16.276086,-26.921050,8.204924,25.747473,581.842044,...,769.0,5.647887e-13,-1.594959,14.364197,-17.006261,6.382874,0.617950,17.483468,-21.293804,9.628577
66,24.168326,50.850000,8.314214,7.372894,1.071057,2.478000,0.134769,0.391668,26.850003,67.908916,...,760.0,9.430345e-14,1.059486,2.320000,0.370000,0.245523,1.689527,4.930000,0.690000,0.806292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19194,19.402535,47.240000,-10.887953,9.491448,1.713780,5.233529,-0.478015,0.983272,33.479192,124.992000,...,761.0,1.147679e-13,1.261722,5.210000,-0.296907,0.611369,2.887820,20.597104,-0.374348,2.831246
19195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19196,22.783616,47.620000,0.508465,8.307436,1.293571,2.752462,0.302960,0.505078,31.283518,91.491000,...,761.0,8.294139e-14,1.146402,2.545001,0.490448,0.371409,1.916890,7.405464,0.690000,1.070630


In [169]:
# could pad/cut to specific length to have full time series
# fit a spline model and extrapolate the parameters and use those terms as the data <--- WORK ON THIS
# frequency domain features (maybe not as useful)
# dynamic time warping (look up)
# use list of features Narayan sends for time-series
# quartiles/ quantiles of the data

In [176]:
time_series_features['output'] = data_and_outcomes['composite_outcome'].fillna(0)
time_series_features = time_series_features.dropna(thresh=20)
time_series_features

/var/folders/yz/gdzsf62s3rjg09h2ns2j9kv00000gn/T/ipykernel_66732/3953143525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_series_features['output'] = data_and_outcomes['composite_outcome'].fillna(0)


,Rf_mean,Rf_max,Rf_min,Rf_std,VT_mean,VT_max,VT_min,VT_std,VE_mean,VE_max,...,PB_std,Ti_mean,Ti_max,Ti_min,Ti_std,Te_mean,Te_max,Te_min,Te_std,output
0,26.343108,50.043451,6.693317,7.409566,0.817306,1.412597,0.212274,0.229081,21.862376,53.535141,...,9.504415e-14,0.995927,1.650559,0.137207,0.219165,1.472634,5.120000,0.297896,0.688078,0.0
4,19.614454,39.811840,6.092655,6.364680,1.437693,2.867007,0.182088,0.818760,31.405264,101.433000,...,9.441178e-14,1.377040,2.725223,0.710000,0.324599,2.017309,6.310680,0.771320,0.892687,0.0
10,58.432143,430.980933,-99.208942,117.091794,-0.112978,12.462207,-18.777080,5.948967,-57.271659,222.796267,...,1.701798e-12,1.103423,12.406791,-2.452976,2.030230,3.013339,16.726619,-4.910225,4.298140,0.0
65,28.868194,273.182742,-80.721037,77.819433,1.109187,16.276086,-26.921050,8.204924,25.747473,581.842044,...,5.647887e-13,-1.594959,14.364197,-17.006261,6.382874,0.617950,17.483468,-21.293804,9.628577,0.0
66,24.168326,50.850000,8.314214,7.372894,1.071057,2.478000,0.134769,0.391668,26.850003,67.908916,...,9.430345e-14,1.059486,2.320000,0.370000,0.245523,1.689527,4.930000,0.690000,0.806292,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19129,28.130446,57.690000,8.953067,10.255420,0.593025,1.397540,0.256980,0.176173,17.077211,37.947000,...,8.460670e-14,1.045445,2.631573,0.370000,0.354377,1.416997,5.172892,0.432063,0.765242,0.0
19133,17.211277,46.510000,-19.173487,6.790479,2.584275,4.566000,-0.198407,1.015591,43.817089,118.897000,...,1.093749e-13,1.370348,9.806264,-1.614586,0.950670,2.624660,14.369006,0.890000,1.570858,0.0
19169,22.120486,53.100000,3.484399,8.736458,1.318394,2.554328,0.029755,0.458215,30.436016,80.018000,...,1.064166e-13,1.277455,5.971501,-0.259734,0.721944,1.971006,8.640904,0.580000,1.201051,0.0
19194,19.402535,47.240000,-10.887953,9.491448,1.713780,5.233529,-0.478015,0.983272,33.479192,124.992000,...,1.147679e-13,1.261722,5.210000,-0.296907,0.611369,2.887820,20.597104,-0.374348,2.831246,0.0


In [177]:
time_series_features['output'].value_counts()

output
0.0    2612
1.0     590
Name: count, dtype: int64

In [178]:
from sklearn.model_selection import train_test_split
#stratify to test and train have same percentage of death
#cross-validation for testing hyperparameters <--- IMPORTANT
X_train, X_test, y_train, y_test = train_test_split(time_series_features[time_series_features.columns.drop('output')].to_numpy(), time_series_features['output'].to_numpy(), test_size=0.2, random_state=1115)

In [179]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, mean_squared_error, confusion_matrix
from tqdm import tqdm
from xgboost import XGBClassifier

In [180]:
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

/opt/miniconda3/envs/cpetai/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [00:44:07] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [181]:
y_pred = model.predict(X_test)

In [182]:
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[496,  23],
       [104,  18]])

In [183]:
tn, fp, fn, tp = conf_mat.ravel()

# Calculate precision
precision = tp / (tp + fp)

# Calculate recall (sensitivity)
recall = tp / (tp + fn)

# Calculate specificity
specificity = tn / (tn + fp)

print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('Specificity: ' + str(specificity))

Precision: 0.43902439024390244
Recall: 0.14754098360655737
Specificity: 0.9556840077071291


In [184]:
values = model.feature_importances_
top_k_features = pd.DataFrame()
feat_coef_dict = dict(zip(time_series_features.columns, values))
top_k = sorted(feat_coef_dict.items(), key=lambda x: abs(x[1]), reverse=True)[0:20]
top_k_features['feature'] = [i[0] for i in top_k]
top_k_features['importance'] = [i[1] for i in top_k]
top_k_features['abs_importance'] = top_k_features['importance'].abs()
top_k_features
#shapley values instead of imprtance

,feature,importance,abs_importance
0,METS_std,0.054703,0.054703
1,METS_max,0.045619,0.045619
2,HR_max,0.020781,0.020781
3,VO2/Kg_max,0.018651,0.018651
4,VCO2_std,0.016077,0.016077
5,O2exp_max,0.015856,0.015856
6,VE/VCO2_mean,0.015634,0.015634
7,VE/VO2_std,0.015529,0.015529
8,CO2exp_std,0.014123,0.014123
9,VO2/HR_std,0.014000,0.014000


,ID,MRN,Test Date,MRN-test date,Death,death_date,death_yrs,pre_arrhythm_surg,pre_arrhythm_surg_date,pre_arrhythm_surg_yrs,...,post_valve_surgery_unspecified,post_valve_surgery_unspecified_date,post_valve_surgery_unspecified_yrs,post_ventricular_fib_flut,post_ventricular_fib_flut_date,post_ventricular_fib_flut_yrs,post_ventricular_tachycardia,post_ventricular_tachycardia_date,post_ventricular_tachycardia_yrs,composite_outcome
1,2.0,84095,2012-09-07,84095-41159,0.0,NaT,NaN,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2014-03-31,1.56,0.0
2,3.0,124420,2017-05-26,124420-42881,0.0,NaT,NaN,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2022-03-31,4.85,0.0
3,4.0,131672,2018-02-02,131672-43133,0.0,NaT,NaN,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2021-08-13,3.53,0.0
4,5.0,157545,2014-09-19,157545-41901,0.0,NaT,NaN,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2014-09-27,0.02,1.0
6,7.0,264481,2012-10-03,264481-41185,1.0,2020-01-15,7.28,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2014-02-03,1.34,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5164,3549.0,99226003,2010-02-12,99226003-40221,0.0,NaT,NaN,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,0.0,NaT,NaN,1.0
5165,3550.0,99246357,2012-05-04,99246357-41033,0.0,NaT,NaN,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,0.0,NaT,NaN,0.0
5166,3551.0,99411555,2010-10-19,99411555-40470,0.0,NaT,NaN,0.0,NaT,NaN,...,0.0,NaT,NaN,0.0,NaT,NaN,0.0,NaT,NaN,0.0
5167,3552.0,99425332,2008-10-15,99425332-39736,0.0,NaT,NaN,NaN,NaT,NaN,...,1.0,2019-02-21,10.35,0.0,NaT,NaN,1.0,2017-12-06,9.14,1.0
